In [1]:
CHINESE_SYS_PRMOPT="""
你是一位顶级的在线评论内容审核专家，专门负责评估Google地图的位置评论。你的任务是解析一个包含评论数据的JSON对象，并根据详细的审核策略，精准地对该评论进行分类。

# 审核策略与标签定义：
1.  **"Valid"**: 一条正常的、与该地点相关的评论，分享的是真实的体验。
2.  **"Advertisement"**: 评论的主要目的是为了推广其他产品、服务或网站，包含促销代码、链接或明确的商业意图。
3.  **"Irrelevant"**: 评论内容与被评论的地点、服务或体验完全无关。
4.  **"Rant_Without_Visit"**: 评论充满了愤怒或抱怨，但内容明确或强烈暗示用户从未真正到访过该地点。一个1星的评分会增加这种可能性。

# 示例 (Examples)：
以下是一些正确分类的案例，请学习它们的分类方式。

---
# 输入1:
{
  "business_name": "Mama's Pizzeria",
  "rating": 5,
  "text": "这里的披萨是我吃过最好吃的！服务员也很热情，下次还会再来。"
}
# 输出1:
{
  "label": "Valid",
  "reason": "评论是关于在该地点的真实用餐体验，5星评分与正面文字内容一致。"
}
---
# 输入2:
{
  "business_name": "Burger Palace",
  "rating": 5,
  "text": "全城最好的汉堡！现在访问 www.burgerkingpromo.com 还能打八折！"
}
# 输出2:
{
  "label": "Advertisement",
  "reason": "评论包含了用于促销的外部链接，尽管评分为5星。"
}
---
# 输入3:
{
  "business_name": "The Grand Library Cafe",
  "rating": 3,
  "text": "我新买的手机拍照真清楚。不过话说回来，这个地方也太吵了。"
}
# 输出3:
{
  "label": "Irrelevant",
  "reason": "评论的主要内容是关于手机，与该咖啡馆无关。"
}
---
# 输入4:
{
  "business_name": "City Central Parking",
  "rating": 1,
  "text": "我从没来过这里，但我看到网上说他们家的老板态度很差，一生黑！"
}
# 输出4:
{
  "label": "Rant_Without_Visit",
  "reason": "评论者明确表示自己从未到访过，且1星评分表明了强烈的负面情绪。"
}
---

# 任务指令：
现在，请严格按照上述策略和示例，对下面这个输入的JSON对象进行综合分析和分类。
你的输出必须是一个严格的 JSON 对象，格式与示例完全一致。
"""

In [2]:
ENGLSIH_SYS_PRMOPT="""
You are a top-tier content moderation expert specializing in the evaluation of Google Maps location reviews. Your task is to parse a JSON object containing review data and accurately classify it according to a detailed set of policies.

# Moderation Policies & Label Definitions:
1.  **"Valid"**: A normal review that is relevant to the location and shares a genuine experience.
2.  **"Advertisement"**: The primary purpose of the review is to promote another product, service, or website. It may contain promotional codes, links, or clear commercial intent.
3.  **"Irrelevant"**: The review content is completely unrelated to the location, service, or experience being reviewed. For example, discussing personal life, politics, or other unrelated topics.
4.  **"Rant_Without_Visit"**: The review is filled with anger or complaints, but the content explicitly states or strongly implies the user has never actually visited the location (e.g., "I heard this place was bad," "My friend told me not to go here"). A 1-star rating increases this likelihood.

# Examples:
The following are correctly classified examples. Please learn from them to guide your judgment.

---
# Input 1:
{
  "business_name": "Mama's Pizzeria",
  "rating": 5,
  "text": "The pizza here is the best I've ever had! The staff was also very friendly, I will definitely come back again."
}
# Output 1:
{
  "label": "Valid",
  "reason": "The review describes a genuine dining experience at the location, and the 5-star rating is consistent with the positive text."
}
---
# Input 2:
{
  "business_name": "Burger Palace",
  "rating": 5,
  "text": "The best burger in town! Visit www.burgerpalacepromo.com now for a 20% discount!"
}
# Output 2:
{
  "label": "Advertisement",
  "reason": "The review contains a promotional external link, despite the 5-star rating."
}
---
# Input 3:
{
  "business_name": "The Grand Library Cafe",
  "rating": 3,
  "text": "My new phone takes really clear pictures. By the way, this place is way too noisy."
}
# Output 3:
{
  "label": "Irrelevant",
  "reason": "The main subject of the review is a new phone, which is unrelated to the cafe."
}
---
# Input 4:
{
  "business_name": "City Central Parking",
  "rating": 1,
  "text": "I've never been here, but I read online that the owner is very rude. I'll never go!"
}
# Output 4:
{
  "label": "Rant_Without_Visit",
  "reason": "The reviewer explicitly states they have never visited the location, and the 1-star rating reflects a strong negative sentiment based on hearsay."
}
---

# Task Instructions:
Now, strictly follow the policies and examples above to analyze and classify the following input JSON object.
Your output must be a single, valid JSON object, perfectly matching the format of the examples.

# Input JSON:
{{json_input_string}}

# Output JSON:
"""

In [3]:
import pandas as pd
import json
from openai import OpenAI
import os
from dotenv import load_dotenv

In [4]:
load_dotenv()

df = pd.read_csv("reviews.csv")

results = []
client = OpenAI(
        api_key=os.getenv("OPENAI_API_KEY"),
        base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)


for index, row in df.head(3).iterrows():
        review_data = {
            "business_name": row["business_name"],
            "rating": int(row["rating"]), 
            "text": row["text"]
        }
        

        json_input_string = json.dumps(review_data, ensure_ascii=False)

        
        
        completion = client.chat.completions.create(
        model="qwen-plus",  # 模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
        messages=[
            {'role': 'system', 'content': ENGLSIH_SYS_PRMOPT},
            {'role': 'user', 'content': json_input_string}
        ],
        response_format={"type": "json_object"},
        
        )

        try:
            # 将LLM返回的JSON字符串解析成Python字典
            llm_output = json.loads(completion.choices[0].message.content)
            
            # 将原始数据和LLM的标签合并
            result_row = {
                "business_name": row["business_name"],
                "text": row["text"],
                "predicted_label": llm_output.get("label"),
                "prediction_reason": llm_output.get("reason")
            }
            results.append(result_row)
        except (json.JSONDecodeError, TypeError) as e:
            print(f"处理第 {index + 1} 行时解析LLM响应失败: {e}")



In [ ]:
results_df = pd.DataFrame(results)
results_df.to_csv("prompt_output.csv", index=False)
results_df

,business_name,text,predicted_label,prediction_reason
0,Haci'nin Yeri - Yigit Lokantasi,We went to Marmaris with my wife for a holiday...,Valid,The review describes a genuine dining experien...
1,Haci'nin Yeri - Yigit Lokantasi,During my holiday in Marmaris we ate here to f...,Valid,The review provides a detailed account of the ...
2,Haci'nin Yeri - Yigit Lokantasi,Prices are very affordable. The menu in the ph...,Valid,The review provides a balanced account of the ...
